In this notebook we will look at rewokring some of our existing functions to pull from newly created google cloud stored csv files as opposed to directly accessing our databases. We will also modify our database populator script in the end to update these locally stored csv files as well as our databases.

In [17]:
# first we will look at adding some new functions that can be inpout to our final data script
# these functions will take our created and modified dataframes that are created when we update our data
# the output will be all of our desired csv files.

def make_csvs(X_evaluated, X_from_db, gains_df):
    '''
    This function intakes our fully completed dataframes from each section of our database populator and writes them to csv files.
    '''
    X_evaluated.to_csv('./CSVs/predictions.csv', index_label='Date')
    X_from_db.to_csv('./CSVs/performance.csv', index_label='Date')
    gains_df.to_csv('./CSVs/quantitative_stats.csv', index_label='Date')
    
    print("CSV's created!")

In [8]:
# we will not bring in our database tables and write them to csv files to be uploaded to the cloud for use in our streamlit app
import pandas as pd
import sqlite3
from sqlite3 import Error
def get_tables_for_csvs():
    '''
    This function recalls our database tables and inputs them into pandas for writing to csv files
    '''
    
    conn = None
    try:
        conn = sqlite3.connect('bitcoin_model.db')
        print('Connected Successfully!')
    
    except Error as e:
        print(e)
        
    query1 = """
            SELECT *
            FROM predictions
            """
    query2 = """
            SELECT *
            FROM performance
            """
    query3 = """
            SELECT *
            FROM quantitative_stats
            """
    
    predictions = pd.read_sql(query1, conn, parse_dates=['Date'])        # adding in the query, connection, and parsing date column as its correct format
    predictions.set_index('Date', inplace=True)                          # setting the index of the resulting dataframe to the date column
    print('Got Predictions')
    performance = pd.read_sql(query2, conn, parse_dates=['Date'])
    performance.set_index('Date', inplace=True)
    print('Got Performance')
    quant_stats = pd.read_sql(query3, conn, parse_dates=['Date'])
    quant_stats.set_index('Date', inplace=True)
    print('Got Quantitative Stats')
    conn.close()                                                         # closing connection to database
    
    return predictions, performance, quant_stats

In [9]:
predictions, performance, quant_stats = get_tables_for_csvs()

Connected Successfully!
Got Predictions
Got Performance
Got Quantitative Stats


In [12]:
quant_stats

,Entry,Win,Exit,Pct_Change,Gains(%),Stake_In,Stake_Out,Net_Profits,Profit_YTD,ROI(%)
Date,,,,,,,,,,
2021-12-06,50441.92,1,50588.00,0.002896,0.002896,1021.148737,1024.105988,2.957251,24.105988,0.024106
2021-12-05,49396.33,1,50441.00,0.021149,0.021149,1000.000000,1021.148737,21.148737,21.148737,0.021149
2021-12-07,50588.95,1,50471.33,0.002325,0.002325,1024.105988,1026.487048,2.381060,26.487048,0.026487
2021-12-08,50471.19,0,47917.53,0.050596,-0.050596,1026.487048,974.550510,-51.936539,-25.449490,-0.025449
2021-12-09,47545.59,1,47668.85,0.002592,0.002592,974.550510,977.076992,2.526482,-22.923008,-0.022923


In [18]:
make_csvs(predictions, performance, quant_stats)

CSV's created!


In [14]:
# now we also need a function to read all of these csv's from our files for when we want to update our website

def get_csvs():
    predictions = pd.read_csv('https://storage.googleapis.com/my_perfect_model_data/CSVs/predictions.csv', index_col='Date', parse_dates=True)
    performance = pd.read_csv('https://storage.googleapis.com/my_perfect_model_data/CSVs/performance.csv', index_col='Date', parse_dates=True)
    quant_stats = pd.read_csv('https://storage.googleapis.com/my_perfect_model_data/CSVs/quantitative_stats.csv', index_col='Date', parse_dates=True)
    
    return predictions, performance, quant_stats

In [15]:
preds, perf, quant = get_csvs()

In [16]:
preds

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base volume,Taker buy quote volume,High/Low,volX,quote-buy,7sma,30sma,90sma,Prediction,Confidence,True_Label,Correct_Pred
Date,,,,,,,,,,,,,,,,,,,
2021-12-05,49152.46,49699.05,47727.21,49396.33,45580.82012,2.231486e+09,1371839.0,22420.90324,1.097786e+09,1971.84,48956.690555,48962.601856,54363.010000,59211.006333,55064.792000,1,0.545161,1,1
2021-12-06,49396.32,50891.11,47100.00,50441.92,58571.21575,2.851529e+09,1741956.0,28820.34753,1.403465e+09,3791.11,48684.816982,48697.007398,53315.248571,58843.383333,55104.549667,1,0.707422,1,1
2021-12-07,50441.91,51936.33,50039.74,50588.95,38253.46877,1.950574e+09,1212350.0,18679.11099,9.525511e+08,1896.59,50990.761438,50995.524794,52406.447143,58420.562000,55155.001222,1,0.597826,1,1
2021-12-08,50588.95,51200.00,48600.00,50471.19,38425.92466,1.925459e+09,1118225.0,18126.84362,9.084107e+08,2600.00,50108.326713,50114.113151,51447.464286,57852.074000,55200.290667,1,0.765154,0,0
2021-12-09,50471.19,50797.76,47320.00,47545.59,37692.68665,1.841475e+09,1130599.0,18097.05960,8.840825e+08,3477.76,48854.967968,48852.275411,50171.071429,57205.338333,55230.231556,1,0.845105,1,1


In [ ]:
# this works great for accessing our csv files, will be using this in our streamlit deployment

In [23]:
%run predictor.py

*********************



The price of Bitcoin is predicted to go UP tomorrow!
I am quite confident about this!
At this confidence I am correct 59.55% of the time!



This is not finanical advice. I am not a financial advisor. All information here is for entertainment purposes only.



*********************
